In [30]:
import thecypher as cy
import urllib.request as urllib2
from bs4 import BeautifulSoup
import pandas as pd
import re
from unidecode import unidecode
import re
from urllib.request import urlopen
import requests
import time
import lyricsgenius
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import numpy
import csv
import matplotlib.pyplot as plt
import random
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import logging
from io import StringIO
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score

In [26]:
df = pd.read_csv("data.csv")
df.head()
df = df[pd.notnull(df['lyrics'])]

col = ['genre', 'lyrics']
df = df[col]

df.columns = ['genre', 'lyrics']
df['genre_id'] = df['genre'].factorize()[0]

genre_id_df = df[['genre', 'genre_id']].drop_duplicates().sort_values('genre_id')
genre_to_id = dict(genre_id_df.values)
id_to_genre = dict(genre_id_df[['genre_id', 'genre']].values)

df.head()

# df_wordCount.dropna(
#     axis=0,
#     how='any',
#     thresh=None,
#     subset=None,
#     inplace=True
# )

# genres = ['pop','death-metal','rock','r&b/soul','hip-hop/rap','alternative','metal','punk','pop/rock',]


# N = 200 # number of records to pull from each genre
# RANDOM_SEED = 200 # random seed to make results repeatable

# train_df = pd.DataFrame()
# test_df = pd.DataFrame()
# for g in genres: # loop over each genre
#     subset = df_wordCount[df_wordCount.genre == g] # create subset
#     print(len(subset))
#     train_set = subset.sample(n=N, random_state=RANDOM_SEED)
#     test_set = subset.drop(train_set.index)
#     train_df = train_df.append(train_set) # append subsets to the master sets
#     test_df = test_df.append(test_set)

,genre,lyrics,genre_id
0,pop,summer high school first would makeout mustang...,0
1,pop,yeah yeah yeah feel phoenix inside march alone...,0
2,pop,told dreams started laughin guess mind actuall...,0
3,pop,lost today would stay could love enough stimul...,0
4,pop,nice legs daisy dukes makes go way like lowcut...,0


In [27]:
# # define our model
# wordCount_clf = Pipeline(
#     [('vect', CountVectorizer()),
#      ('clf', LogisticRegression(random_state=0))])

# # train our model on training data
# wordCount_clf.fit(train_df.lyrics, train_df.genre)  

# # score our model on testing data
# predicted = wordCount_clf.predict(test_df.lyrics)
# print(np.mean(predicted == test_df.genre))

train_df = shuffle(train_df)
test_df = shuffle(test_df)

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(df.lyrics).toarray()
labels = df.genre_id
features.shape

(3288, 8642)

In [29]:
# # define our model
# text_clf = Pipeline(
#     [('vect', TfidfVectorizer()),
#      ('clf', LogisticRegression(random_state=0))])

# # train our model on training data
# text_clf.fit(train_df.lyrics, train_df.genre)  

# # core our model on testing data
# predicted = text_clf.predict(test_df.lyrics)
# np.mean(predicted == test_df.genre)
X_train, X_test, y_train, y_test = train_test_split(df['lyrics'], df['genre'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = LogisticRegression().fit(X_train_tfidf, y_train)

In [32]:
accuracy = cross_val_score(LogisticRegression(random_state=0), features, labels, scoring='accuracy', cv=2)
accuracy

array([0.39902676, 0.40085158])